In [ ]:
# pip install pajek-tools

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
# from pajek_tools import PajekWriter
# import igraph as ig

%matplotlib inline


In [ ]:
df = pd.read_csv('../input/music-artists-popularity/artists.csv')
df.head()

## **TIỀN XỬ LÝ DỮ LIỆU**

In [ ]:
df.info()

**Vì dữ liệu quá lớn nên chỉ lấy 200 ca sĩ**


In [ ]:
df1 = df.head(200)

In [ ]:
df1.to_csv('artist_small.csv')

In [ ]:
df1.info()

In [ ]:
# Tách các tag trong cột Tags_lastfm thành nhiều dòng
lastfm = df1.set_index(df1.columns.drop('tags_lastfm',1).tolist()).tags_lastfm.str.split(';', expand=True).stack().reset_index().rename(columns={0:'tags_lastfm'}).loc[:, df1.columns]

In [ ]:
lastfm.head()

In [ ]:
lastfm.to_csv('lastfm1.csv')

In [ ]:
# lastfm[lastfm['mbid']=='cc197bad-dc9c-440d-a5b5-d52ba2e14234']

In [ ]:
lastfm.info()

**vì dữ liệu lớn nên chỉ lấy 50 tag có số bài hát nhiều nhất**

In [ ]:
#vì dữ liệu lớn nên chỉ lấy 50 tag có số bài hát nhiều nhất
lastfm.groupby("tags_lastfm").agg({ "mbid": pd.Series.nunique}).sort_values(by="mbid",ascending=False).head(50)


In [ ]:

lastfm = lastfm.groupby('tags_lastfm').filter(lambda x : x['mbid'].count() >= 30)
# lastfm.to_csv('a.csv')

In [ ]:
lastfm_final = lastfm[['tags_lastfm','artist_lastfm' ]]
lastfm_final

In [ ]:
lastfm_final=lastfm_final.drop_duplicates()

# **Network Analysis**

### **Xây dựng network**

### **Ban đầu là đồ thị 2 phía**

Node: là tag(tags_lastfm) và nghệ sĩ(artist_lastfm), 
Edge: 1 tag được nối với nhiều nghệ sĩ, 1 nghệ sĩ được nối với nhiều tag

In [ ]:
# convert data frame sang graph
lastfm_graph_2phia = nx.from_pandas_edgelist(lastfm_final, source='tags_lastfm', target='artist_lastfm')

In [ ]:
lastfm_graph_2phia.graph

In [ ]:
# Lập danh sách các tags, chúng ta sẽ sử dụng nó sau
tags = list(lastfm['tags_lastfm'].unique())
len(tags)

In [ ]:
# Lập danh sách những ca sĩ, 
artists = list(lastfm['artist_lastfm'].unique())
len(artists)

In [ ]:
dict(zip(artists, artists))


In [ ]:
node_labels = dict(zip(tags, tags))

Vẽ đồ thị 2 phía, nhưng chỉ gắn label cho tag

In [ ]:
pos = nx.spring_layout(lastfm_graph_2phia)
# tô màu các nút theo phân vùng của chúng
# cmap = cm.get_cmap('rainbow', max(partition.values()) + 1)
plt.figure(figsize=(12, 10))

nx.draw_networkx_nodes(lastfm_graph_2phia, pos, 
                      #  partition.keys(), 
                       node_size=300,
                      #  cmap=cmap,
                       node_color='lightblue'
                       )

nx.draw_networkx_nodes(lastfm_graph_2phia, pos, nodelist=artists, node_color='#cccccc', node_size=5)

nx.draw_networkx_edges(lastfm_graph_2phia, pos, alpha=0.5)

# chỉ gắn label cho tag
nx.draw_networkx_labels(lastfm_graph_2phia, pos, font_size=10, labels=node_labels)

plt.show()

### Chuyển đổi thành đồ thị 1 phía

Node: là nghệ sĩ(artist_lastfm), 
Edge: các nghệ sĩ được nối với nhau nếu cùng tag

In [ ]:
nodes_0 = list(set(lastfm_final['artist_lastfm'].values.tolist()))

nodes_1 = list(set(lastfm_final['tags_lastfm'].values.tolist()))

edges = lastfm_final.values.tolist()
# edges

In [ ]:
from networkx.algorithms import bipartite

lastfm_graph_2 = nx.Graph()
lastfm_graph_2.add_nodes_from(nodes_0, bipartite=0) # Add the node attribute “bipartite”
lastfm_graph_2.add_nodes_from(nodes_1, bipartite=1)
lastfm_graph_2.add_edges_from(edges,weight=1)

lastfm_graph_2.edges(data=True)

#convert đồ thì 2 phía thành 1 phía
lastfm_graph = nx.projected_graph(lastfm_graph_2, nodes_0)

In [ ]:
df_new = pd.DataFrame(lastfm_graph)
df_new.to_csv('df_new.csv')

In [ ]:
print("Nodes:" + str(lastfm_graph.number_of_nodes()))
print("Edges:" + str(lastfm_graph.number_of_edges()))

In [ ]:
nx.write_gexf(lastfm_graph, "lastfm.gexf")

**Xem thử ca sĩ Taylor Swwift kết nối với bao nhiêu nghệ sĩ có cùng thể loại nhạc**

In [ ]:
lastfm_graph.degree('Taylor Swift')

In [ ]:
# [artists for tag in artists]

In [ ]:
[lastfm_graph.degree(artist) for artist in artists]


In [ ]:

plt.figure(figsize=(12, 12))

# Tạo layout cho các nút 
layout = nx.spring_layout(lastfm_graph,iterations=50)


artist_size = [lastfm_graph.degree(artist) * 10 for artist in artists]
nx.draw_networkx_nodes(lastfm_graph, 
                       layout, 
                       nodelist=artists, 
                       node_size=artist_size, # a LIST of sizes, based on g.degree
                       node_color='lightblue')


nx.draw_networkx_edges(lastfm_graph, layout, width=1, edge_color="#cccccc")

node_labels = dict(zip(artists, artists))
nx.draw_networkx_labels(lastfm_graph, layout, labels=node_labels)

plt.axis('off')

plt.title("Graph Artist")

plt.show()

### Phát hiện cộng đồng dựa vào Louvain và girvan newman 

In [ ]:
from community import community_louvain


# Tìm cộng đồng các nút 
partition = community_louvain.best_partition(lastfm_graph, resolution = 1)
values = list(partition.values())


# Kiểm tra số lượng cộng đồng đã được tạo
print('Number of communities:', len(np.unique(values)))

Chúng ta xem thử trong cụm thứ nhất có những nghệ sĩ nào?

In [ ]:
search_k = 0
for name, k in partition.items(): 
    if k == search_k:
        print(name)

In [ ]:
pos = nx.spring_layout(lastfm_graph)
# tô màu các nút theo phân vùng của chúng
cmap = cm.get_cmap('rainbow', max(partition.values()) + 1)
plt.figure(figsize=(12, 10))

nx.draw_networkx_nodes(lastfm_graph, pos, partition.keys(), node_size=300,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(lastfm_graph, pos, alpha=0.5)
nx.draw_networkx_labels(lastfm_graph, pos, font_size=10)

plt.show()

Ta phát hiện được 3 cộng đồng,  cụm màu xanh là những nghệ sĩ thuộc thế hệ cũ, thập niên 90-20s;
cụm màu đỏ là những nghệ sĩ thuộc thế hệ pop mới đang hoạt động sôi nổi hiện nay, họ thuộc top nghệ sĩ được đông đảo nhiều người hâm mộ;
cụm màu tím là những nghệ sĩ thuộc thế hệ rock, những thể loại thuộc underground đang hoạt động sôi nổi tuy nhiên nhiều nghệ sĩ chưa nổi và phù hợp với giới underground, trẻ thích sự nổi loạn


In [ ]:
from networkx.algorithms.community.centrality import girvan_newman
communities = girvan_newman(lastfm_graph)

tuple(sorted(c) for c in next(communities))

In [ ]:
 degcen = nx.degree_centrality(lastfm_graph)
 sorted_degcen = sorted(degcen.items(), key=lambda x:x[1], reverse=1)[0:10]
 sorted_degcen

In [ ]:
 clocen = nx.closeness_centrality(lastfm_graph)
 sorted_clocen = sorted(clocen.items(), key=lambda x:x[1], reverse=1)[0:10]
 sorted_clocen

In [ ]:
pr = nx.pagerank(lastfm_graph)
sorted_pr = sorted(pr.items(), key=lambda x:x[1], reverse=1)[0:10]
sorted_pr

In [ ]:
betcen = nx.betweenness_centrality(lastfm_graph)
sorted_betcen = sorted(betcen.items(), key=lambda x:x[1], reverse=1)[0:10]
sorted_betcen